<a href="https://colab.research.google.com/github/Vakhranev/Gramota/blob/main/%D0%A1%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D0%B8%20%D0%B2%20XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xml.etree.ElementTree as ET
import re

# Список сокращений
abbreviations = set([
    "адыг.", "америк.", "англ.", "арабск.", "арамейск.", "ассир.", "атт.", "баскск.", "библ.",
    "болг.", "букв.", "валлийск.", "венг.", "вост.", "в-т", "галльск.", "гаек.", "гаэльск.", "герм.",
    "гот.", "греч.", "груз.", "дагест.", "дат.", "диал.", "др.", "евр.", "егип.", "ж.", "зап.", "им.",
    "ирл.", "исл.", "исп.", "ит.", "кавк.", "карфагенск.", "кельт.", "коптск.", "корейск.", "коре.",
    "кумык.", "ласк.", "лат.", "лит.", "м.", "марийск.", "н.", "нар.", "нем.", "нидерл.", "нов.",
    "норв.", "норм.", "орф.", "осет.", "перс.", "пожелат.", "половецк.", "польск.", "порт.", "прил.",
    "прованс.", "разг.", "римск.", "румынск.", "рус.", "савойск.", "санскр.", "сербск.", "сирийск.",
    "сканд.", "слав.", "см.", "сокр.", "ср.", "стар.", "тат.", "традиц.", "тур.", "тюрк.", "укр.",
    "уменьш.", "усеч.", "ф.", "финск.", "флам.", "фракийск.", "фр.", "фриз.", "халдейск.", "церк.",
    "цыг.", "чешек.", "швед.", "шотл.", "эльз.", "эолийск.", "др.-евр."
])

# Функция обработки XML
def process_xml(input_file, output_file):
    tree = ET.parse(input_file)
    root = tree.getroot()

    page_number = 1
    new_pages = []

    for page in root.findall('.//page'):
        new_page = ET.Element('page', {'number': str(page_number)})
        page_number += 1

        paragraphs = [p.text.strip() for p in page.findall('.//paragraph') if p.text]
        text = " ".join(paragraphs)

        if text == "ЖЕНСКИЕ ИМЕНА":
            new_page.append(ET.Element('female_names'))
            new_page[-1].text = text
            new_pages.append(new_page)
            continue
        elif text == "МУЖСКИЕ ИМЕНА":
            new_page.append(ET.Element('male_names'))
            new_page[-1].text = text
            new_pages.append(new_page)
            continue

        words = text.split()
        i = 0

        while i < len(words):
            if words[i][0].isupper():
                word = words[i]
                i += 1

                definition_words = []
                open_square_brackets = 0
                open_parentheses = 0

                while i < len(words):
                    definition_words.append(words[i])

                    # Проверка на открытые скобки
                    open_square_brackets += words[i].count('[')
                    open_square_brackets -= words[i].count(']')

                    # Если встречаем точку, проверяем завершение
                    if words[i][-1] == '.':
                        # Проверка на сокращения
                        if words[i] not in abbreviations and open_square_brackets == 0 and open_parentheses == 0:
                            # Проверка следующего слова
                            if i + 1 < len(words) and words[i + 1][0].isupper():
                                # Если следующее слово с большой буквы, то завершение
                                break
                    i += 1

                # Собираем определение и добавляем его в запись
                definition = " ".join(definition_words).strip()

                entry = ET.Element('entry')
                word_elem = ET.SubElement(entry, 'word')
                word_elem.text = word.rstrip(',')
                def_elem = ET.SubElement(entry, 'definition')
                def_elem.text = definition
                new_page.append(entry)

            i += 1

        new_pages.append(new_page)

    root.clear()
    for new_page in new_pages:
        root.append(new_page)

    tree.write(output_file, encoding="utf-8", xml_declaration=True)

# Входной и выходной файлы
input_file = 'Superanskaya_Slovar_russkikh_lichnykh_imyon.xml'
output_file = 'processed_Slovar.xml'

# Запуск обработки
process_xml(input_file, output_file)

In [ ]:
import xml.etree.ElementTree as ET
import re

def is_cyrillic(word):
    # Убираем точку в конце, если есть
    word = word.rstrip('.')

    # Проверяем, состоит ли слово из заглавных кириллических букв и (cid:141), причем оно должно быть длиннее одной буквы
    return re.fullmatch(r'(?:[А-ЯЁ]+(?:\(cid:141\))?)+', word) is not None and len(word) > 1

def contains_vowel(word):
    # Проверяем, есть ли в слове хотя бы одна гласная (А, Е, Ё, И, О, У, Ы, Э, Ю, Я)
    vowels = "АЕЁИОУЫЭЮЯ"
    return sum(1 for char in word if char in vowels)

def process_xml(input_file, output_file):
    tree = ET.parse(input_file)
    root = tree.getroot()

    new_root = ET.Element("document")
    page_number = 0
    skip_next = False
    current_entry = None
    collecting_definition = False  # Флаг, показывающий, собираем ли мы текст для definition

    for paragraph in root.findall(".//paragraph"):
        text = paragraph.text.strip() if paragraph.text else ""

        # Проверяем, является ли paragraph номером страницы
        if text.isdigit():
            page_number += 1
            skip_next = True  # Устанавливаем флаг, чтобы пропустить следующий paragraph
            continue

        if skip_next:
            skip_next = False
            continue  # Пропускаем этот paragraph

        # Пропускаем параграфы, содержащие только одну букву
        if len(text) == 1:
            continue

        words = text.split()

        # Проверяем, если в paragraph только одно слово
        if len(words) == 1:
            word = words[0]

            # Проверяем, что слово написано в верхнем регистре, без "(cid:141)" и с более чем одной гласной
            if word.isupper() and contains_vowel(word) > 1 and "(cid:141)" not in word:
                continue  # Удаляем этот paragraph, не добавляем его в новый документ

        i = 0
        while i < len(words):
            word = words[i]

            # Проверка, если слово после "см."
            if i > 0 and words[i-1].lower() == "см." and is_cyrillic(word):
                # Добавляем слово в текущий definition
                if collecting_definition and current_entry is not None:
                    definition_elem = current_entry.find("definition")
                    if definition_elem.text:
                        definition_elem.text += " " + word
                    else:
                        definition_elem.text = word
                i += 1
                continue

            if is_cyrillic(word) and contains_vowel(word):  # Нашли новое слово, которое удовлетворяет всем условиям
                # Заканчиваем сбор предыдущего definition (если он есть)
                collecting_definition = False

                # Создаём новую запись
                current_entry = ET.SubElement(new_root, "entry")
                word_elem = ET.SubElement(current_entry, "word")
                word_elem.text = word

                # Создаём новый definition
                definition_elem = ET.SubElement(current_entry, "definition")
                collecting_definition = True

            elif collecting_definition and current_entry is not None:
                # Если это не новое слово, добавляем текст в текущий definition
                definition_elem = current_entry.find("definition")
                if definition_elem.text:
                    definition_elem.text += " " + word
                else:
                    definition_elem.text = word

            i += 1

    new_tree = ET.ElementTree(new_root)
    new_tree.write(output_file, encoding="utf-8", xml_declaration=True)

# Входной и выходной файлы
input_file = 'Glinkina_Etimologicheskie_tayni_russkoy_orfografii.xml'
output_file = 'processed_Glinkina.xml'

# Запуск обработки
process_xml(input_file, output_file)

In [3]:
import xml.etree.ElementTree as ET
import re

def process_xml(input_file, output_file):
    tree = ET.parse(input_file)
    root = tree.getroot()

    new_root = ET.Element("document")
    page_number = 0
    paragraphs = list(root.findall(".//paragraph"))

    # Фильтруем параграфы: удаляем номера страниц и одиночные буквы
    filtered_paragraphs = []
    for para in paragraphs:
        text = para.text.strip()
        if text.isdigit():
            page_number += 1
            continue
        if len(text) == 1 and text.isalpha():
            continue
        filtered_paragraphs.append(text)

    # Объединяем параграфы, если предыдущий не заканчивается на точку
    merged_paragraphs = []
    buffer = ""
    for text in filtered_paragraphs:
        if buffer and not buffer.endswith("."):
            buffer += " " + text
        else:
            if buffer:
                merged_paragraphs.append(buffer)
            buffer = text
    if buffer:
        merged_paragraphs.append(buffer)

    i = 0
    current_definition_text = ""  # Для хранения текста определения
    word_text = ""  # Для хранения текста слова

    while i < len(merged_paragraphs):
        text = merged_paragraphs[i]

        # Регулярка: слово из заглавных букв (включая "(cid:1)"), оканчивающееся запятой, точкой с запятой или запятой и цифрой
        match = re.search(r"^([А-ЯЁ\(cid:1\)]+[,;]\d*)", text)

        if match:
            if current_definition_text:
                entry = ET.Element("entry")
                word_elem = ET.SubElement(entry, "word")
                word_elem.text = word_text.strip()

                if (current_definition_text.strip().split() and word_text.strip().split() and
    word_text.strip().split()[-1] == current_definition_text.strip().split()[-1]):
                    definition_elem = ET.SubElement(entry, "definition")
                    definition_elem.text = ""
                else:
                    definition_elem = ET.SubElement(entry, "definition")
                    definition_elem.text = current_definition_text.strip()

                new_root.append(entry)

            word_text = match.group(1).strip()
            match_full = re.search(r"^([А-ЯЁ\(cid:1\) ,.;-]+)", text)
            if match_full:
                word_text = match_full.group(1).strip()

            current_definition_text = text[len(word_text):].strip()

            while True:
                match_end = re.search(r"\.([ \t\n])", current_definition_text)
                if match_end:
                    if current_definition_text[match_end.end():match_end.end() + 1] == ";":
                        current_definition_text = current_definition_text[match_end.end():]
                        continue

                    if current_definition_text[:match_end.start()].endswith("мн"):
                        word_text += current_definition_text[:match_end.end()]
                        current_definition_text = current_definition_text[match_end.end():]
                        continue

                    next_word = current_definition_text[match_end.end():].strip().split()[0]
                    if next_word[0].islower() or next_word[0].startswith("("):
                        word_text += current_definition_text[:match_end.end()]
                        current_definition_text = current_definition_text[match_end.end():]
                        continue

                    word_text += current_definition_text[:match_end.end()]
                    current_definition_text = current_definition_text[match_end.end():].strip()
                    break
                else:
                    word_text += current_definition_text.strip()
                    break
        else:
            current_definition_text += " " + text.strip()

        i += 1

    if current_definition_text:
        entry = ET.Element("entry")
        word_elem = ET.SubElement(entry, "word")
        word_elem.text = word_text.strip()

        if current_definition_text.strip().split() and word_text.strip().split()[-1] == current_definition_text.strip().split()[-1]:
            definition_elem = ET.SubElement(entry, "definition")
            definition_elem.text = ""
        else:
            definition_elem = ET.SubElement(entry, "definition")
            definition_elem.text = current_definition_text.strip()

        new_root.append(entry)

    new_tree = ET.ElementTree(new_root)
    new_tree.write(output_file, encoding="utf-8", xml_declaration=True)

input_file = "Rakhmanova_Trudnosti_russkogo_yazyka.xml"
output_file = "processed_Rakhmanova.xml"
process_xml(input_file, output_file)

In [34]:
import xml.etree.ElementTree as ET
import re

def remove_spaces_after_commas(input_file):
    # Загружаем XML
    tree = ET.parse(input_file)
    root = tree.getroot()

    # Ищем все параграфы в документе
    paragraphs = root.findall(".//paragraph")

    for paragraph in paragraphs:
        text = paragraph.text.strip()

        # Убираем пробелы после запятой
        text = re.sub(r",\s+", ",", text)

        # Обновляем текст параграфа
        paragraph.text = text

    # Сохраняем изменённый XML
    tree.write(input_file, encoding="utf-8", xml_declaration=True)

# Указываем путь к исходному файлу
input_file = "Shelyakin_Obyasnitelnyi_slovar_neproveryayemykh_orphogramm_russkogo_yazyka.xml"
remove_spaces_after_commas(input_file)

In [36]:
import xml.etree.ElementTree as ET
import re

def process_xml(input_file, output_file):
    tree = ET.parse(input_file)
    root = tree.getroot()

    new_root = ET.Element("document")
    paragraphs = list(root.findall(".//paragraph"))

    # Регулярное выражение для захвата слова с запятой
    word_pattern = re.compile(r"(-)*[A-ZА-ЯЁÁÉÓÍÚÝÄÖÜÑČŠŽ^]+([\/\s,A-ZА-ЯЁÁÉÓÍÚÝÄÖÜÑČŠŽ^-])*")

    entries = []
    definition_text = ""
    inside_parentheses = False

    for i in range(len(paragraphs)):
        paragraph = paragraphs[i]
        text = paragraph.text.strip()

        if len(text.split()) == 1 and text.isupper():
            continue

        if text.isdigit() or (len(text) == 1 and text.isalpha()):
            continue

        if i > 0 and paragraphs[i - 1].text.strip().isdigit():
            next_paragraph_text = text.strip()
            words_in_next_paragraph = next_paragraph_text.split()

            if len(words_in_next_paragraph) >= 2 and all(word.isupper() for word in words_in_next_paragraph[:2]):
                words_in_next_paragraph.pop(0)
                text = ' '.join(words_in_next_paragraph)
                paragraph.text = text

        buffer = ""
        for char in text:
            if char.isspace():
                if buffer:
                    if '(' in buffer:
                        inside_parentheses = True

                    if inside_parentheses:
                        definition_text += " " + buffer
                    elif word_pattern.fullmatch(buffer):
                        if entries:
                            entries[-1]["definition"] += " " + definition_text.strip()
                        definition_text = ""
                        entries.append({"word": buffer, "definition": ""})
                    else:
                        definition_text += " " + buffer

                    if ')' in buffer:
                        inside_parentheses = False
                    buffer = ""
            else:
                buffer += char

        if buffer:
            if '(' in buffer:
                inside_parentheses = True

            if inside_parentheses:
                definition_text += " " + buffer
            elif word_pattern.fullmatch(buffer):
                if entries:
                    entries[-1]["definition"] += " " + definition_text.strip()
                definition_text = ""
                entries.append({"word": buffer, "definition": ""})
            else:
                definition_text += " " + buffer

            if ')' in buffer:
                inside_parentheses = False

    if definition_text and entries:
        entries[-1]["definition"] += " " + definition_text.strip()

    # Выводим данные в новый XML
    for entry in entries:
        entry_elem = ET.Element("entry")
        word_elem = ET.SubElement(entry_elem, "word")
        word_elem.text = entry["word"]
        definition_elem = ET.SubElement(entry_elem, "definition")
        definition_elem.text = entry["definition"].strip()
        new_root.append(entry_elem)

    new_tree = ET.ElementTree(new_root)
    new_tree.write(output_file, encoding="utf-8", xml_declaration=True)

input_file = "Shelyakin_Obyasnitelnyi_slovar_neproveryayemykh_orphogramm_russkogo_yazyka.xml"
output_file = "processed_Shelyakin.xml"
process_xml(input_file, output_file)

In [37]:
import xml.etree.ElementTree as ET
import re

def add_spaces_after_commas(input_file):
    # Загружаем XML
    tree = ET.parse(input_file)
    root = tree.getroot()

    # Ищем все параграфы в документе
    paragraphs = root.findall(".//paragraph")

    for paragraph in paragraphs:
        text = paragraph.text.strip()

        # Добавляем пробел после каждой запятой, если его нет
        text = re.sub(r",(?!\s)", ", ", text)

        # Обновляем текст параграфа
        paragraph.text = text

    # Сохраняем изменённый XML
    tree.write(input_file, encoding="utf-8", xml_declaration=True)

# Указываем путь к файлу
input_file = "processed_Shelyakin.xml"
add_spaces_after_commas(input_file)